In [ ]:
# Snowflake-META Onboarding Example
# This notebook demonstrates how to onboard control table specs for Snowflake using Snowpark

import sys
import os
import json
import pandas as pd
from snowflake.snowpark import Session
from snowflake.snowpark.functions import col, lit
from snowflake.snowpark.types import StructType, StructField, StringType, VariantType

# Add the src directory to the path
#sys.path.append(os.path.join(os.getcwd(), '..', 'src'))

from src.onboard_controltable import OnboardControlTable
from src.controltable_spec import ControlTableSpecUtils

print("Snowflake-META Onboarding Example with Snowpark")
print("=" * 50)


In [ ]:
# Step 1: Get Snowpark Session from Notebook Warehouse
# The session is automatically tied to the warehouse assigned to this notebook
# No need for config files or manual connection setup

# Get the current Snowpark session
session = Session.builder.getOrCreate()
print("Snowpark session established successfully!")
print(f"Current warehouse: {session.get_current_warehouse()}")
print(f"Current database: {session.get_current_database()}")
print(f"Current schema: {session.get_current_schema()}")




In [ ]:
# Step 2: Define onboarding parameters
onboarding_params_map = {
    "database": "RAW",
    "schema":"SNOWMETA_CONFIG",
    "onboarding_file_path":'@"RAW"."SNOWMETA_CONFIG"."MYSTAGE"/onboarding/sample_onboarding_metadata.json',
    "bronze_control_table": "sample_bronze_control_table", 
    "silver_control_table": "sample_silver_control_table",
    "overwrite": "True",
    "env": "dev",
    "version": "v1",
    "import_author": "Maeruf"
}

print("Onboarding parameters configured:")
for key, value in onboarding_params_map.items():
    print(f"  {key}: {value}")

In [ ]:
df_onboarding_file = session.read.json(onboarding_params_map['onboarding_file_path'])
df_onboarding_file

In [ ]:
# Step 4: Initialize and run the onboarding process

# Create the OnboardControlTable instance using Snowpark session
onboard_control_table = OnboardControlTable(
    session=session,
    dict_obj=onboarding_params_map,
    bronze_schema_mapper=None,  # Optional: custom schema mapper
    horizon_enabled=True  # Set to True if using Horizon
)

print("OnboardControlTable instance created successfully!")

# Run the onboarding process
try:
    print("Starting onboarding process...")
    onboard_control_table.onboard_controltable_specs()
    print("✅ Onboarding completed successfully!")
except Exception as e:
    print(f"❌ Onboarding failed: {str(e)}")
    raise
